In [ ]:
# hide
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
import httpx
import pandas as pd

from urllib.parse import urljoin

from will_it_saturate import servers
from will_it_saturate import clients
from will_it_saturate.epochs import Epoch
from will_it_saturate.hosts import Host, HostDetails

from will_it_saturate.results import Result
from will_it_saturate.files import BenchmarkFile
from will_it_saturate.control.client import ControlClient
from will_it_saturate.repositories import SqliteRepository, register_default_tables

<IPython.core.display.Javascript object>

# Create Benchmark Config

In [ ]:
# dont_test

repository = SqliteRepository.build_repository("results_remote.db")
register_default_tables(repository)
results_table = repository.tables["result"]

control_server_port, server_port = 8100, 5100
server_host_name = "staging.wersdoerfer.de"
server_control_host = Host(name=server_host_name, port=control_server_port)
server_control_client = ControlClient(host=server_control_host)
server_details = server_control_client.get_host_details()

client_control_host = Host(name="wersdoerfer.de", port=control_server_port)
client_control_client = ControlClient(host=client_control_host)
client_details = client_control_client.get_host_details()

servers_to_test = [
    servers.FastAPIUvicornServer(host=server_host_name, port=server_port),
    servers.DjangoGunicornWSGIServer(host=server_host_name, port=server_port),
    servers.NginxDockerServer(host=server_host_name, port=server_port),
]
clients_to_test = [
    clients.HttpxClient(name="httpx"),
    clients.AioHttpClient(name="aiohttp"),
    clients.WrkClient(name="wrk", host=server_host_name, port=server_port),
]
epochs_to_test = [
    Epoch(file_size=10 ** 7, duration=10),
]

<IPython.core.display.Javascript object>

In [ ]:
# dont_test
# single server
# to_test_server = servers.FastAPIUvicornServer(host=server_host_name, port=server_port)
# to_test_server = servers.DjangoGunicornWSGIServer(
#     host=server_host_name, port=server_port
# )
# server = server_control_client.get_or_create_server(to_test_server)
# epoch = Epoch(file_size=10 ** 4, duration=10)
# epoch.files = server_control_client.get_or_create_files(epoch)
# epoch.urls = [server.file_to_url(file) for file in epoch.files]
# for epoch_file in epoch.files:
#     epoch_file.port = server.port
#     epoch_file.hostname = server.host

<IPython.core.display.Javascript object>

In [ ]:
# benchmark_client = clients.WrkClient(
#     name="wrk", host=server_host_name, port=server_port
# )
# client_control_client.measure(benchmark_client, epoch)

<IPython.core.display.Javascript object>

In [ ]:
# server.stop()

<IPython.core.display.Javascript object>

In [ ]:
# dont_test


def prepare_epoch(server_control_client, server, epoch):
    epoch.files = server_control_client.get_or_create_files(epoch)
    epoch.urls = [server.file_to_url(file) for file in epoch.files]
    for epoch_file in epoch.files:
        epoch_file.port = server.port
        epoch_file.hostname = server.host
    return epoch


def measure(benchmark_client, epoch, server, server_details, client_details):
    result = Result(
        server=server,
        client=benchmark_client,
        server_details=server_details,
        client_details=client_details,
        file_size=epoch.file_size,
        complete_size=epoch.complete_size,
    )
    result.elapsed = client_control_client.measure(benchmark_client, epoch)
    result_id = results_table.add_result(result)
    return result


for server in servers_to_test:
    server = server_control_client.get_or_create_server(server)
    for epoch in epochs_to_test:
        epoch = prepare_epoch(server_control_client, server, epoch)
        for benchmark_client in clients_to_test:
            result = measure(
                benchmark_client, epoch, server, server_details, client_details
            )
            result_id = results_table.add_result(result)
    server_control_client.stop_server(server)

ConnectError: [Errno 61] Connection refused

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

results_from_database = results_table.get_results()

cols = set(["server", "client", "elapsed", "file_size_h", "bytes_per_second_h"])
df = pd.DataFrame(
    [
        {k: v for k, v in r.dict_for_pandas().items() if k in cols}
        for r in results_from_database
    ]
)
df.sort_values(by="bytes_per_second_h", ascending=False)

,server,client,elapsed,file_size_h,bytes_per_second_h
4,fastAPI/uvicorn,wrk,17.562824,9.54MB,67.88MB
5,fastAPI/uvicorn,wrk,17.562824,9.54MB,67.88MB
10,django/gunicorn/wsgi,wrk,18.326388,9.54MB,65.05MB
11,django/gunicorn/wsgi,wrk,18.326388,9.54MB,65.05MB
12,nginx/docker,httpx,19.666250,9.54MB,60.62MB
13,nginx/docker,httpx,19.666250,9.54MB,60.62MB
0,fastAPI/uvicorn,httpx,20.408017,9.54MB,58.41MB
1,fastAPI/uvicorn,httpx,20.408017,9.54MB,58.41MB
2,fastAPI/uvicorn,aiohttp,20.919098,9.54MB,56.99MB
3,fastAPI/uvicorn,aiohttp,20.919098,9.54MB,56.99MB


<IPython.core.display.Javascript object>

## Create Benchmark Server

create server control client + server (needed for turning files into urls)

In [ ]:
# dont_test

server_control_host = Host(name="staging.wersdoerfer.de", port=8100)
server_control_client = ControlClient(host=server_control_host)
server_details = server_control_client.get_host_details()
# server = server_control_client.get_or_create_server(
#     servers.FastAPIUvicornServer(host="192.168.178.113", port=5001)
# )
server = server_control_client.get_or_create_server(
    servers.NginxDockerServer(host="192.168.178.113", port=5001)
)
# server = server_control_client.get_or_create_server(
#     servers.DjangoGunicornWSGIServer(host="staging.wersdoerfer.de", port=5001)
# )

<IPython.core.display.Javascript object>

## Create Files and Urls

In [ ]:
# dont_test

file_size, duration = 10 ** 7, 10
epoch = Epoch(file_size=file_size, duration=duration)
epoch.files = server_control_client.get_or_create_files(epoch)
epoch.urls = [server.file_to_url(file) for file in epoch.files]

<IPython.core.display.Javascript object>

## Modify Files

In [ ]:
# dont_test

for epoch_file in epoch.files:
    epoch_file.port = server.port
    epoch_file.hostname = server.host

<IPython.core.display.Javascript object>

## Create Control Client

In [ ]:
# dont_test

client_control_host = Host(name="wersdoerfer.de", port=8001)
client_control_client = ControlClient(host=client_control_host)
client_details = client_control_client.get_host_details()
# benchmark_client = clients.HttpxClient(name="httpx")
# benchmark_client = clients.AioHttpClient(name="aiohttp")
benchmark_client = clients.WrkClient(
    name="wrk ethernet", host=server.host, port=server.port
)

<IPython.core.display.Javascript object>

## Create Result

In [ ]:
# dont_test

result = Result(
    server=server,
    client=benchmark_client,
    server_details=server_details,
    client_details=client_details,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

<IPython.core.display.Javascript object>

In [ ]:
# epoch.urls[:5]

<IPython.core.display.Javascript object>

In [ ]:
# result.elapsed = 2.80
# result.readable_bytes_per_second

<IPython.core.display.Javascript object>

## Measure Server with Client

In [ ]:
# dont_test

result.elapsed = client_control_client.measure(benchmark_client, epoch)

<IPython.core.display.Javascript object>

## Stop Server

In [ ]:
# dont_test

server_control_client.stop_server(server)

True

<IPython.core.display.Javascript object>

## Save Result

We are using a sqlite database for saving benchmark results.

In [ ]:
# dont_test

repository = SqliteRepository.build_repository("results_remote.db")
register_default_tables(repository)
results_table = repository.tables["result"]

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result_id = results_table.add_result(result)

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

results_from_database = results_table.get_results()

cols = set(["server", "client", "elapsed", "file_size_h", "bytes_per_second_h"])
pd.DataFrame(
    [
        {k: v for k, v in r.dict_for_pandas().items() if k in cols}
        for r in results_from_database
    ]
)

""


<IPython.core.display.Javascript object>